In [2]:
import sys
sys.path.insert(0, '../')

import importlib
import coralme
import coralme.solver.solver
import coralme.builder.main
import coralme.core.model
import cobra
from coralme.util.helpers import get_nlp,optimize

#model = cobra.io.load_json_model
me = coralme.io.pickle.load_pickle_me_model('../species_files/Zymomonas_mobilis/outputs/MEModel-step3-zymomonas-TS.pkl')
nlp = get_nlp(me)
rxn_index_dct = {r.id : me.reactions.index(r) for r in me.reactions}
met_index_dct = {m.id : me.metabolites.index(m) for m in me.metabolites}

In [3]:
%%time
sol,basis = optimize(rxn_index_dct,met_index_dct,nlp,max_mu = 0.1, min_mu = 0., maxIter = 100,
		tolerance = 1e-6, precision = 'quad', verbose = True, basis = None)

Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        0	0.1000000000000000	Optimal
CPU times: user 25.1 s, sys: 91.8 ms, total: 25.2 s
Wall time: 25.1 s


In [ ]:
# loop through carbons, shut off all but one, try to run
carbon_exchanges = [
    'EX_gcald_e', 'EX_glc_e', 'EX_glcn_e', 'EX_glyc_e', 
    'EX_succ_e', 'EX_sucr_e', 'EX_fum_e', 'EX_fru_e', 
    'EX_g3pc_e', 'EX_g3pe_e', 'EX_g3pg_e', 'EX_g3pi_e', 
    'EX_g3ps_e', 'EX_gal_e', 'EX_gal__bD_e', 'EX_gdp_e', 
    'EX_gln__L_e', 'EX_glu__L_e', 'EX_glyc3p_e', 'EX_lac__D_e', 
    'EX_xyl__D_e', 'EX_glycogen'
]
for run in carbon_exchanges:
    # turn off everything
    for exch in carbon_exchanges:
        me.reactions.get_by_id(exch).upper_limit = 0
        me.reactions.get_by_id(exch).lower_limit = 0

    # turn back on run
    me.reactions.get_by_id(run).upper_limit = 0
    me.reactions.get_by_id(run).lower_limit = 0
    
    # now run the model
    nlp = get_nlp(me)
    rxn_index_dct = {r.id : me.reactions.index(r) for r in me.reactions}
    met_index_dct = {m.id : me.metabolites.index(m) for m in me.metabolites}
    sol,basis = optimize(rxn_index_dct,met_index_dct,nlp,max_mu = 0.1, min_mu = 0., maxIter = 100,
		tolerance = 1e-6, precision = 'quad', verbose = True, basis = None)
    break

In [4]:
sol

,fluxes,reduced_costs
biomass_dilution,1.000000e-01,1.403950e-02
protein_biomass_to_biomass,5.448134e-02,-6.340615e-36
mRNA_biomass_to_biomass,1.831993e-04,0.000000e+00
tRNA_biomass_to_biomass,1.105558e-03,0.000000e+00
rRNA_biomass_to_biomass,7.632764e-03,1.781177e-36
...,...,...
TS_for_c,2.773743e-03,0.000000e+00
TS_cobalt2_c,-6.279887e-07,0.000000e+00
TS_zn2_c,-2.615265e-07,0.000000e+00
TS_mn2_c,-8.949355e-08,0.000000e+00


In [33]:
print('Carbon fluxes currently on')
for flux, val in sol.fluxes.items():
    if 'EX_' in flux and val != 0:
        contains_C = False
        for metab in me.reactions.get_by_id(flux).metabolites.keys():
            if 'C' in metab.formula:
                contains_C = True
        if contains_C:
            print(flux, end = ' : ')
            print(val)

print('\nCarbon fluxes currently off')
for flux, val in sol.fluxes.items():
    if 'EX_' in flux and val == 0:
        contains_C = False
        for metab in me.reactions.get_by_id(flux).metabolites.keys():
            if 'C' in metab.formula:
                contains_C = True
        if contains_C:
            print(flux, end = ' : ')
            print(val)

Carbon fluxes currently on
EX_3dhq_e : 0.015304163535025436
EX_ac_e : 0.0438275071381385
EX_ca2_e : 2.8576910686740527e-29
EX_cgly_e : -3.908782637386756e-58
EX_cl_e : -7.328967445100167e-58
EX_co2_e : 138.4190932428587
EX_cu_e : -2.6914116521208935e-29
EX_etoh_e : 138.39359818005826
EX_g15l_e : 1.2508104439637619e-56
EX_gcald_e : -2.373585726756539e-27
EX_glc_e : -70.0
EX_glcn_e : -8.222332248043147e-23
EX_glyc_e : 6.254052219818809e-57
EX_gthrd_e : 3.908782637386756e-58
EX_gtp_e : -7.817565274773512e-58
EX_inost_e : -1.5635130549547023e-57
EX_pnto__R_e : -3.162309421149475e-35
EX_succ_e : 0.024401955272469713
EX_sucr_e : -2.224562111460849e-57
EX_fum_e : 0.067969339196368

Carbon fluxes currently off
EX_3dhsk_e : 0.0
EX_4hbz_e : 0.0
EX_acald_e : 0.0
EX_actn__S_e : 0.0
EX_ade_e : 0.0
EX_anhgm_e : 0.0
EX_arg__L_e : 0.0
EX_asn__L_e : 0.0
EX_asp__L_e : 0.0
EX_btn_e : 0.0
EX_butso3_e : 0.0
EX_cd2_e : 0.0
EX_chol_e : 0.0
EX_cobalt2_e : 0.0
EX_cynt_e : 0.0
EX_etha_e : 0.0
EX_ethso3_e : 0.0
